# Data pipeline 

## 1.1 Carga de Información

Ya que hemos configurado el manejador de base de datos ahora crearemos la DB, llamemosla `Transacciones`, como requerimos de las claves del service account para realizar la conexión no tenemos un gran riesgo de seguridad al dejar explícita la contraseña y el usser.  

In [ ]:
import psycopg2 # conector a PostgreSQL
import pandas as pd # manioulacion de data.frames
import numpy as np # operaciones vectorcizadas en C :D 
import gc # grabage collector para liberar memoria explicitamente 

Revisemos cómo vienen los datos crudos en el .csv, observando los primeros y últimos 50 registros. 

In [ ]:
! wc -l data_prueba_tecnica.csv # para conocer el numero de lineas del dataset original

In [ ]:
cargos_head = pd.read_csv('data_prueba_tecnica.csv', nrows=50, error_bad_lines=True, warn_bad_lines =True, skip_blank_lines=False)
cargos_tail = pd.read_csv('data_prueba_tecnica.csv', nrows=50, skiprows=9950, header=None, \
                           names=list(cargos_head.columns), error_bad_lines=True, warn_bad_lines =True, skip_blank_lines=False)
cargos = cargos_head.append( cargos_tail)#, ignore_index=True)
cargos.head(100)

In [ ]:
# como ya no requerimos estos dataframes los eliminamos para liberar memoria 
del cargos_head
del cargos_tail
gc.collect()

En general los primeros y últimos registros lucen bien con excepción de de las líneas en blanco alternadas con cada registro con información, veamos cómo viene el dataset completo.


In [ ]:
cargos_row = pd.read_csv('data_prueba_tecnica.csv', error_bad_lines=True, warn_bad_lines =True, skip_blank_lines=True, \
                        encoding = 'utf-8', verbose =True, infer_datetime_format=True,  parse_dates= ['created_at', 'paid_at'])
print(cargos_row.shape)
cargos_row.head(20)

Para sorpresa, no hay errores ni warnings en la lectura de los datos crudos.


## 1.2 Transformación

In [ ]:
print(cargos_row.describe(include='all'))
print('-------------------')
print(cargos_row.shape)
print(cargos_row.isna().sum()) #number of on.nas

De lo anterior es importante notar los siguientes puntos:
- Existen 3 registros con `id` y `name` nulos, al igual que 4 `company_id`.
- El campo `id` parece ser de buena calidad después de arreglar el punto anterior.
- Existen 4 valores para el nombre de compañías diferentes que habrá que validar.
- También en la limpieza tendremos que considerar la distribución de los valores para la columna `amount` y detectar valores atípicos. 

Finalmente validaremos que la fecha de actualización de los registros sea posterior a la fecha de creación del registro. 


### Limpieza de nulos 

In [ ]:
# vamos a seleccionar los registros con nulos para limpiarlos
def Selec_na_index( data ): 
    '''
    Inputs 
     data (pandas data.frame): Solo con las columnas de interes
    Outputs:
     index (numpy array):  Indices de data donde existen nulos 
    '''
    columnas = data.columns
    index =  [ list( data[x].index[ pd.isna( data[x] ) ])  for x in columnas ]
    index = sum( index, []) 
    index.sort()
    return( np.array(index))

Como los registros con `id` nulo contienen información les asignamos uno para no perder esa información __posteriormente revisaremos el porqué estos registros ‘llegarón’ con  `id` nulo__ 

In [ ]:
index = Selec_na_index( cargos_row[['id', 'name', 'company_id', 'amount', 'status', 'created_at']])
nulos = cargos_row.iloc[index]
index = cargos_row['id'].index[ pd.isna( cargos_row['id'])]
cargos_row['id'][index] = range(0, len(index))

Para limpiar las columnas `name` y `company_id` nos valdremos de la misma información contenida en el dataset, construiremos un catálogo y sustituiremos los  nulos y valores atípicos. 


In [ ]:
catalogo = cargos_row[['name', 'company_id']].drop_duplicates()
catalogo

El catálogo nos permite ver que estamos en una situación sencilla todos los valores nulos y  anormales para `name` y `company_id` son asociados al valor `MiPasajefy`. Por lo que será sencillo limpiarlos, __sin embargo valdría la pena investigar el pipeline que produce este archivo ya que es curioso que solo se esté presentando para esta compañía__ 

In [ ]:
index = cargos_row['name'].index[ pd.isna( cargos_row['name'])]
cargos_row['name'][index] = catalogo.name[0]
index = cargos_row['company_id'].index[ pd.isna( cargos_row['company_id'])]
cargos_row['company_id'][index] = catalogo.company_id[0]
cargos_row = cargos_row.replace({'name':  {x : catalogo.name[0] for x in list(set(cargos_row.name)) if x not in ['Muebles chidos', 'MiPasajefy'] } } )
cargos_row = cargos_row.replace({'company_id': { x : catalogo.company_id[0] for x in list(set(cargos_row.company_id))  if x not in ['8f642dc67fccf861548dfe1c761ce22f795e91f0', 'cbf1c8b09cd5b549416d49d220a40cbd317f952e'] } } )

### Errores de encoding y validación de fechas 

Notamos que la columna`status` tiene errores de encoding, uno que puede identificarse a simple vista y otro que sustituiremos por un valor ‘Desconocido’, para no perder esta transacción y nuevamente __valdría la pena investigar el proceso que extrae este archivo porque este valor de encoding se presenta en la misma compañía en la que hemos tenido problemas__. 

In [ ]:
print(set(cargos_row.status))
cargos_row = cargos_row = cargos_row.replace({'status':  { 'p&0x3fid' : 'paid',  '0xFFFF': 'desconocido'}})

In [ ]:
a = list(set(cargos_row.status))
for i in a: 
    print(i)
    print(i.encode().decode())
    

In [ ]:
'a'.encode()